In [3]:
import pandas as pd
import urllib


In [36]:
def build_df(year, month, day, hour):
    """
    returns a pandas dataframe with columns:
        project, article, views, content_returned
    based on input date string:
        e.g, "2016", "06", "24", "01"
    files are found at
        https://dumps.wikimedia.org/other/pagecounts-all-sites/2016/2016-05/pagecounts-20160501-000000.gz
    """
    base_url_string = "https://dumps.wikimedia.org/other/pagecounts-all-sites/"
    url_string = base_url_string + "/" + year + "/" + year + "-" + month
    url_string += "/pagecounts-" + year + month + day + "-" + hour + "0000.gz"
    
    file_name = year + month + day + "-" + hour + ".gz"
    
    # download file
    opener = urllib.URLopener()    
    opener.retrieve(url_string, file_name)
    

    # return data frame
    df = pd.read_csv(file_name, compression="gzip", sep=" ", 
                 error_bad_lines=False, header=None)
    df.columns = ['project', 'article', 'views', 'content_returned']
    # include only en-wiki
    df = df[df['project'] == 'en']
    df = df[['article', 'views']]
    
    return df

def combine_dataframe(df1, df2):
    """
    sums the page views in the given dataframes
        uses an outer join (treating nan as 0)
    returns a new dataframe
        'article', 'views'
    """
    df_combined = df1.merge(df2, left_on='article', right_on='article', how='outer')
    df_combined.fillna(0, inplace=True)
    df_combined['views'] = df_combined['views_x'] + df_combined['views_y']
    df_combined = df_combined[['article', 'views']]
    df_combined = df_combined.drop_duplicates('article')
    df_combined.set_index('article', inplace=True)
    
    return df_combined

In [37]:
%%timeit
df1 = build_df("2016", "06", "01", "01")

1 loop, best of 3: 1min 17s per loop


In [8]:
df1.head()

,article,views
1112122,NaN,987
1112123,!,13
1112124,!!,1
1112125,!!!,4
1112126,!!!Fuck_You!!!,2


In [11]:
df2 = build_df("2016", "06", "02", "01")

In [12]:
df_combined = df1.merge(df2, left_on='article', right_on='article', how='outer')

In [20]:
df_combined.fillna(0, inplace=True)

In [21]:
df_combined['views'] = df_combined['views_x'] + df_combined['views_y']

In [26]:
df_c = combine_dataframe(df1, df2)

In [27]:
df_c.head()

,article,views
0,0,1874
1,0,1000
2,0,888
3,0,14
4,0,902


In [32]:
df_c.drop_duplicates('article').describe()

,views
count,4124634.000000
mean,3.170189
std,177.316497
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,351289.000000


In [34]:
df_c.drop_duplicates('article').to_csv("test_csv.csv")

In [35]:
df_c.drop_duplicates('article')

,article,views
0,0,1874
6,!,14
7,!!,1
8,!!!,13
9,!!!Fuck_You!!!,3
10,!!!_(album),3
11,!!Destroy-Oh-Boy!!,1
12,!!Fuck_you!!,1
13,!!Que_Corra_La_Voz!!,1
14,!!_(chess),1
